<a href="https://colab.research.google.com/github/aparnashenoy/ml/blob/master/smote_cvs_resp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip 

     / 56.8MB 5.3MB/s
     |████████████████████████████████| 81kB 5.1MB/s 
     |████████████████████████████████| 604kB 30.9MB/s 
     |████████████████████████████████| 3.1MB 33.3MB/s 
     |████████████████████████████████| 71kB 9.3MB/s 
     |████████████████████████████████| 296kB 45.4MB/s 
  Created wheel for pandas-profiling: filename=pandas_profiling-2.10.0-py2.py3-none-any.whl size=232425 sha256=a9cd7d2034b18990cd797a97849e001e46540bc2f5a9acf9bceae5d30a0a5c3b
  Stored in directory: /tmp/pip-ephem-wheel-cache-1jrx_tx9/wheels/56/c2/dd/8d945b0443c35df7d5f62fa9e9ae105a2d8b286302b92e0109
  Created wheel for htmlmin: filename=htmlmin-0.1.12-cp36-none-any.whl size=27085 sha256=2861d99a6663514c9c06775ba1c4a9ee943375f289c3de4c2debbf7307a43923
  Stored in directory: /root/.cache/pip/wheels/43/07/ac/7c5a9d708d65247ac1f94066cf1db075540b85716c30255459
Successfully built pandas-profiling htmlmin
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uni

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer

%matplotlib inline

In [3]:
path = '/gdrive/My Drive/'

In [4]:
cvs_resp = pd.read_csv(path + 'data_cvs_resp.csv')

In [5]:
cvs_resp.drop(['Unnamed: 0'], axis=1, inplace=True)

In [6]:
cvs_resp.sample(5)

,HR,O2Sat,SBP,MAP,DBP,Resp,Age,Gender,SepsisLabel
854879,88.0,98.0,151.0,97.00,74.00,18.0,87.00,0,0
716258,106.0,94.0,138.0,85.33,61.00,25.0,25.72,0,0
331534,91.0,98.0,120.0,74.00,61.00,16.0,75.52,0,0
1004361,64.0,98.0,97.5,64.00,50.00,18.0,80.00,1,0
133139,83.0,99.0,102.0,71.50,58.75,16.0,59.41,0,0


In [8]:
X = cvs_resp.drop('SepsisLabel', axis=1)
y = cvs_resp.SepsisLabel

In [9]:
#Train-Test split using stratified random sampling
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, stratify=y, random_state=4)

In [10]:
#Checking the class (percentage) distribution of the target in the training set
pd.crosstab(y_train, columns='count')/len(y_train)*100

col_0,count
SepsisLabel,
0,98.078677
1,1.921323


In [11]:
#Checking the class (percentage) distribution of the target in the training set
pd.crosstab(y_test, columns='count')/len(y_test)*100

col_0,count
SepsisLabel,
0,98.078598
1,1.921402


In [14]:
import imblearn

In [15]:
from imblearn import under_sampling, over_sampling

In [16]:
#Performing random undersampling on the training dataset
rus = imblearn.under_sampling.RandomUnderSampler(sampling_strategy=1.0)
X_rus, y_rus = rus.fit_resample(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [17]:
#Checking the class distribution of the target
pd.crosstab(y_rus, columns='count')/len(y_rus)*100

col_0,count
row_0,
0,50.0
1,50.0


In [18]:
#Performing random undersampling on the training dataset
rus = imblearn.under_sampling.RandomUnderSampler(sampling_strategy=1.0)
X_rus_test, y_rus_test = rus.fit_resample(X_test, y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [19]:
#Checking the class distribution of the target
pd.crosstab(y_rus_test, columns='count')/len(y_rus_test)*100

col_0,count
row_0,
0,50.0
1,50.0


In [20]:
from sklearn.naive_bayes import GaussianNB # I am using Gaussian algorithm from Naive Bayes

# Lets creat the model
sepsis_model = GaussianNB()

sepsis_model.fit(X_rus, y_rus.ravel())

GaussianNB(priors=None, var_smoothing=1e-09)

In [21]:
sepsis_train_predict = sepsis_model.predict(X_rus)

from sklearn import metrics

print("Model Accuracy: {0:.4f}".format(metrics.accuracy_score(y_rus, sepsis_train_predict)))
print()

Model Accuracy: 0.5887



In [22]:
sepsis_test_predict = sepsis_model.predict(X_rus_test)

from sklearn import metrics

print("Model Accuracy: {0:.4f}".format(metrics.accuracy_score(y_rus_test, sepsis_test_predict)))
print()

Model Accuracy: 0.5872



In [23]:
print("Confusion Matrix")

print("{0}".format(metrics.confusion_matrix(y_rus_test, sepsis_test_predict, labels=[1, 0])))
print("")

print("Classification Report")
print(metrics.classification_report(y_rus_test, sepsis_test_predict, labels=[1, 0]))

Confusion Matrix
[[3623 3105]
 [2450 4278]]

Classification Report
              precision    recall  f1-score   support

           1       0.60      0.54      0.57      6728
           0       0.58      0.64      0.61      6728

    accuracy                           0.59     13456
   macro avg       0.59      0.59      0.59     13456
weighted avg       0.59      0.59      0.59     13456



In [24]:
from sklearn.ensemble import RandomForestClassifier
sepsis_rf_model = RandomForestClassifier(random_state=52)
sepsis_rf_model.fit(X_rus, y_rus.ravel())

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=52, verbose=0,
                       warm_start=False)

In [25]:
rf_train_predict = sepsis_rf_model.predict(X_rus)
print("Model Accuracy: {0:.2f}".format(metrics.accuracy_score(y_rus, rf_train_predict)))

Model Accuracy: 0.99


In [26]:
rf_test_predict = sepsis_rf_model.predict(X_rus_test)
print("Model Accuracy: {0:.2f}".format(metrics.accuracy_score(y_rus_test, rf_test_predict)))

Model Accuracy: 0.64


In [27]:
print("Confusion Matrix")
print(metrics.confusion_matrix(y_rus_test, rf_test_predict, labels=[1, 0]))
print("")
print("Classification Report")
print(metrics.classification_report(y_rus_test, rf_test_predict, labels=[1, 0]))

Confusion Matrix
[[4325 2403]
 [2414 4314]]

Classification Report
              precision    recall  f1-score   support

           1       0.64      0.64      0.64      6728
           0       0.64      0.64      0.64      6728

    accuracy                           0.64     13456
   macro avg       0.64      0.64      0.64     13456
weighted avg       0.64      0.64      0.64     13456



In [28]:
from sklearn.linear_model import LogisticRegression

sepsis_lr_model = LogisticRegression(C=0.7, random_state=52)
sepsis_lr_model.fit(X_rus, y_rus.ravel())
lr_test_predict = sepsis_lr_model.predict(X_rus_test)

print("Model Accuracy: {0:.2f}".format(metrics.accuracy_score(y_rus_test, lr_test_predict)))
print("")
print("Confusion Matrix")
print(metrics.confusion_matrix(y_rus_test, lr_test_predict, labels=[1, 0]))
print("")
print("Classification Report")
print(metrics.classification_report(y_rus_test, lr_test_predict, labels=[1, 0]))

Model Accuracy: 0.59

Confusion Matrix
[[3699 3029]
 [2473 4255]]

Classification Report
              precision    recall  f1-score   support

           1       0.60      0.55      0.57      6728
           0       0.58      0.63      0.61      6728

    accuracy                           0.59     13456
   macro avg       0.59      0.59      0.59     13456
weighted avg       0.59      0.59      0.59     13456

